In [15]:
import cv2
import numpy as np

In [16]:
#YOLO 로드
net = cv2.dnn.readNet('cross-hands.weights','cross-hands.cfg')
layer_names = net.getLayerNames()
with open("cross-hands.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
output_layers = [layer_names[i[0]- 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=100)

In [32]:
path = 'C:/Python/YGL_final_project/Pictures-resized/20210716_145925.jpg'
img = cv2.imread(path)
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

In [28]:
blob = cv2.dnn.blobFromImage(img, 0.00392,(320,320), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

In [29]:
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
            

In [30]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

In [31]:
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        color = colors[i]
        cv2.rectangle(img, (x,y), (x + w, y + h), color, 2)

cv2.imshow('Image', img)
cv2.imwrite('sample-detect-hands2.jpg', img)
cv2.waitKey(0)
cv2.destroyAllWindows()